In [4]:
import os
from glob import glob
import pandas as pd
import numpy as np

In [5]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
data_dir = '/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/ground_weather_data'
save_dir = '/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/data/'

os.makedirs(save_dir, exist_ok=True)

dirNames = os.listdir(data_dir)
print(dirNames)

['데이터값_설명.txt', '.DS_Store', 'asos188', 'aws779', 'aws793', 'aws885', 'aws792', 'aws863', 'aws330', 'aws884', 'aws893', 'asos184', 'aws865', 'aws781', 'aws329', 'aws328']


In [27]:
from datetime import datetime, tzinfo
from dateutil import tz
import pytz
import time

# 한 시간 단위의 누적 강수량으로 데이터를 변환한다.
def rain_sum(df):
    prevStr = ''
    timeStr = ''
    sum = 0.0

    time_list = []
    hour_rain = []

    for i in range(0, len(df)):
        timeStr = df.iloc[i,0].split(':')[0]
        # 이전 행과 현재 행의 '년.월.일 시'문자열이 같다면 
        # 같은 '시'에 해당하므로 누적으로 더 해줍니다. 
        if prevStr==timeStr:
            sum+=df.iloc[i,1]

        # 문자열이 다르다면 '시'가 변했으므로 누적된 값을 리스트에 저장하고
        # sum값을 새롭게 누적하기 시작합니다.
        else:
            time_list.append(df.iloc[i,0])
            hour_rain.append(sum)
            sum = df.iloc[i,1] 

        #print(df.iloc[i,0],sum)
        prevStr = timeStr
    return time_list, hour_rain


## _rain.csv파일 한 시간 단위로 데이터 주기 변경

In [28]:
import re

for dirName in dirNames:
    csv_list = glob(os.path.join(data_dir,dirName,'*.csv'))
    for csvFile in csv_list:
        # 파일명에 rain을 포함하는 파일을 찾습니다.
        count = csvFile.find('rain')
        if count>0:
            fileName = os.path.basename(csvFile)
            print(csvFile)
            
            # 파일 읽기
            df = pd.read_csv(csvFile, encoding='cp949')
            fileFront = fileName.split('_')[0]
            filterStr = re.sub(r'[0-9]', '',fileFront)
            if filterStr=='aws':
                # 지점 컬럼 삭제(불필요)
                df.drop(columns='지점', inplace=True)
                
            # 결측치를 0으로 채워준다.
            df.fillna(0, inplace=True)
            
            # 한 시간단위로 강수량을 수정한다.
            time_list, hour_rain = rain_sum(df)
            
            # pandas DataFrame 생성
            value = {'일시':time_list, '강수량':hour_rain}
            
            # 시간당 강수량을 저장하는 데이터프레임 생성
            df2 = pd.DataFrame(value)
            
            # 데이터형을 시간 타입으로 변경
            df2['일시'] = pd.to_datetime(df2['일시'])
            
            # 데이터 수집 기간을 2020-3-1 0:00 ~ 2021-1-1 0:00
            df2 = df2[df2['일시']>'2020-3-1 8:00']

            df2.set_index(keys='일시', inplace=True, drop=True)
            df2.to_csv(save_dir+fileName, mode='w', encoding='utf-8-sig')

In [29]:
def oneHour_avg(df):
    prevStr = ''
    timeStr = ''

    temp_sum = 0.0
    wdir_sum = 0.0
    wspd_sum = 0.0
    humi_sum  = 0.0
    
    time_list = []
    temp_list = []
    wdir_list = []
    wspd_list = []
    humi_list = []

    for i in range(0, len(df)):
        timeStr = df.iloc[i,0].split(':')[0]
        # 이전 행과 현재 행의 '년.월.일 시'문자열이 같다면 
        # 같은 '시'에 해당하므로 누적으로 더 해줍니다. 
        if prevStr==timeStr:
            temp_sum += df.iloc[i,1]
            wdir_sum += df.iloc[i,2]
            wspd_sum += df.iloc[i,3]
            humi_sum += df.iloc[i,4]

        # 문자열이 다르다면 '시'가 변했으므로 누적된 값을 리스트에 저장하고
        # sum값을 새롭게 누적하기 시작합니다.
        else:
            time_list.append(df.iloc[i,0])
            temp_list.append(temp_sum/6)
            wdir_list.append(wdir_sum/6)
            wspd_list.append(wspd_sum/6)
            humi_list.append(humi_sum/6)
            temp_sum = df.iloc[i,1] 
            wdir_sum = df.iloc[i,2]
            wspd_sum = df.iloc[i,3]
            humi_sum = df.iloc[i,4]
            
        prevStr = timeStr
    
    return time_list, temp_list, wdir_list, wspd_list, humi_list    

In [30]:
for dirName in dirNames:
    csv_list = glob(os.path.join(data_dir,dirName,'*.csv'))
    for csvFile in csv_list:
        
        count = csvFile.find('rain')
        
        # 파일명에 rain을 포함하지 않는 파일을 찾습니다.
        if count<0:
            fileName = os.path.basename(csvFile)
            #print(csvFile)
            
            # 파일 읽기
            df = pd.read_csv(csvFile, encoding='cp949')

            # 결측치를 0으로 채워준다.
            df.fillna(0, inplace=True)
            # 한 시간단위로 강수량을 수정한다.
            time_list, temp_list, wdir_list, wspd_list, humi_list = oneHour_avg(df)

            # pandas DataFrame 생성
            value = {'일시':time_list, '기온':temp_list, '풍향':wdir_list, '풍속':wspd_list, '습도':humi_list}            
            # 시간당 강수량을 저장하는 데이터프레임 생성
            df2 = pd.DataFrame(value)            

            df2['일시'] = pd.to_datetime(df2['일시'])
            df2 = df2[df2['일시']>'2020.3.1 8:00']
            #print(df2)
            df2.set_index(keys='일시', inplace=True, drop=True)
            df2.to_csv(save_dir+fileName, mode='w', encoding='utf-8-sig')


In [31]:
# UTC Time -> SEOUL Time
def convTime(dt_str):
    timeFormat = "%Y-%m-%d %H:%M"
    
    # Create datetime object in local timezone
    dt_utc = datetime.strptime(dt_str, timeFormat)
    dt_utc = dt_utc.replace(tzinfo=pytz.UTC)
    
    # Get local timezone
    local_zone = tz.tzlocal()

    # Convert timezone of datetime from UTC to local
    dt_local = dt_utc.astimezone(local_zone)
    conv_time = dt_local.strftime(timeFormat)    
    return conv_time

## 위성에서 추출한 일사량 데이터 전처리하기

In [32]:
data_dir = '/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/sat_data'
save_dir = '/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/data/'

dirNames = os.listdir(data_dir)
print(dirNames)

['강정동.csv', '구좌읍.csv', '남원읍.csv', '노형동.csv', '대정읍.csv', '도련이동.csv', '도련일동.csv', '도순동.csv', '도평동.csv', '동홍동.csv', '봉개동.csv', '삼양일동.csv', '상예동.csv', '상효동.csv', '색달동.csv', '서귀동.csv', '서호동.csv', '서홍동.csv', '성산읍.csv', '아라일동.csv', '아라이동.csv', '안덕면.csv', '애월읍.csv', '영평동.csv', '오등동.csv', '오라이동.csv', '오라일동.csv', '외도일동.csv', '용강동.csv', '용담이동.csv', '월평동.csv', '이도일동.csv', '이호이동.csv', '조천읍.csv', '중문동.csv', '토평동.csv', '표선면.csv', '하예동.csv', '하원동.csv', '한경면.csv', '한림읍.csv', '해안동.csv', '호근동.csv', '화북이동.csv', '회천동.csv']


In [33]:
print(len(dirNames))

45


In [34]:
data_dir = './sat_data/'
save_dir = './data/'

csv_list = glob(os.path.join(data_dir,'*.csv'))

for csvFile in csv_list:
    
    # 파일 읽기
    df = pd.read_csv(csvFile, encoding='cp949')
    dropCol = ['Unnamed: 1', 'dim_y', 'dim_x', 'ASR', 
               'ASR_DQF1', 'RSR', 'RSR_DQF1', 'SW_DQF',
               'gk2a_imager_projection']
    
    df.drop(columns = dropCol, inplace=True)
    
    # 결측치를 0으로 채워준다.
    df.fillna(0, inplace=True)
    
    # UTC -> Seoul Time으로 변경한다.
    conv_time = list(map(convTime, df['date']))
    df['date']=conv_time
    # 컬럼명을 수정한다. 'date'-> '일시'
    df.rename(columns = {'date':'일시'},inplace=True)
    df['일시'] = pd.to_datetime(df['일시'])
    df.set_index(keys='일시', inplace=True, drop=True)
    # 파일명만을 가져온다.
    fileName = os.path.basename(csvFile)
    df.to_csv(save_dir+fileName, mode='w', encoding='utf-8-sig')

## 동별 데이터를 생성하기 위한 매칭 정보 읽기

In [35]:
dirNames = os.listdir('./')
dirNames

['.config', 'drive', 'sample_data']

In [36]:
matchFile ='/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/jeju_geo_match_info.csv'

# 파일 읽기
match_df = pd.read_csv(matchFile, encoding='UTF-8')
match_df

,위도,경도,동이름,유형,지점번호,가까운 지점,대체위도,대체경도
0,33.221816,126.252994,대정읍,aws,793,대정,33.2410,126.2263
1,33.240160,126.380600,하예동,aws,328,중문,33.2494,126.4060
2,33.246950,126.489270,강정동,aws,884,기상(과),33.2593,126.5176
3,33.246790,126.563960,서귀동,aws,884,기상(과),33.2593,126.5176
4,33.261770,126.386780,상예동,aws,328,중문,33.2494,126.4060
5,33.264093,126.412000,색달동,aws,328,중문,33.2494,126.4060
6,33.262002,126.428862,중문동,aws,328,중문,33.2494,126.4060
7,33.256966,126.473104,도순동,aws,865,오등,33.4627,126.5191
8,33.262659,126.460024,하원동,aws,884,기상(과),33.2593,126.5176
9,33.259596,126.519646,서호동,aws,884,기상(과),33.2593,126.5176


In [37]:
dropCol = ['위도', '경도', '대체위도', '대체경도', '가까운 지점']
match_df.drop(columns = dropCol, inplace=True)

In [38]:
match_df

,동이름,유형,지점번호
0,대정읍,aws,793
1,하예동,aws,328
2,강정동,aws,884
3,서귀동,aws,884
4,상예동,aws,328
5,색달동,aws,328
6,중문동,aws,328
7,도순동,aws,865
8,하원동,aws,884
9,서호동,aws,884


In [39]:
fileList1 = []
fileList2 = []
for i in range(len(match_df)):
    fileList1.append(match_df.iloc[i, 1]+str(match_df.iloc[i, 2])+'.csv')
    fileList2.append(match_df.iloc[i, 1]+str(match_df.iloc[i, 2])+'_rain.csv')
match_df['파일명1'] = fileList1
match_df['파일명2'] = fileList2
match_df

,동이름,유형,지점번호,파일명1,파일명2
0,대정읍,aws,793,aws793.csv,aws793_rain.csv
1,하예동,aws,328,aws328.csv,aws328_rain.csv
2,강정동,aws,884,aws884.csv,aws884_rain.csv
3,서귀동,aws,884,aws884.csv,aws884_rain.csv
4,상예동,aws,328,aws328.csv,aws328_rain.csv
5,색달동,aws,328,aws328.csv,aws328_rain.csv
6,중문동,aws,328,aws328.csv,aws328_rain.csv
7,도순동,aws,865,aws865.csv,aws865_rain.csv
8,하원동,aws,884,aws884.csv,aws884_rain.csv
9,서호동,aws,884,aws884.csv,aws884_rain.csv


현재 문제는 하향일사량 데이터의 갯수가 중간에 누락되는 부분이다.

## 세 개의 csv파일 합치기

In [41]:
data_dir = '/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/data/'
save_dir = '/content/drive/MyDrive/[2022]AICE자격증반/AICE예제/일사량예측/merge/'

os.makedirs(save_dir, exist_ok=True)

for i in range(0, len(match_df)):
    # 하향 단파 복사
    mergeFile  = data_dir + match_df.iloc[i, 0]+'.csv'
    merge_df   = pd.read_csv(mergeFile, encoding='UTF-8')
    
    # 강수량 정보 파일 : '파일명2'
    precipFile = data_dir + match_df.iloc[i, 4]
    precip_df  = pd.read_csv(precipFile, encoding='UTF-8')

    # 기온, 풍향, 풍속, 습도 정보 파일 : '파일명1'
    awsFile = data_dir + match_df.iloc[i, 3]
    aws_df  = pd.read_csv(awsFile, encoding='UTF-8')
    
    # 데이터 프레임간 합치기
    # 위성 일사량 데이터가 중간 중간 누락되었다.
    # 따라서 AWS,ASOS데이터도 함께 삭제 해주어야 한다.
    # 그래서 데이터프레임을 합치는데 기준을 일사량 데이터의 인덱스로 한다.
    merge_df = pd.merge(merge_df, precip_df, how='left', left_on='일시', right_on='일시')
    merge_df = pd.merge(merge_df, aws_df, how='left', left_on='일시', right_on='일시')
   
    #merge_df.set_index(keys='일시', inplace=True, drop=True)
    merge_df.to_csv(save_dir + match_df.iloc[i, 0]+'.csv', mode='w', encoding='utf-8-sig')


FileNotFoundError: ignored

## 

In [24]:
solarPowerFile = '한국전력거래소_시간별 육지 제주 태양광 및 풍력발전량_2014년-2020년.csv'

# 파일 읽기
power_df = pd.read_csv(solarPowerFile, encoding='cp949')
power_df

FileNotFoundError: ignored

In [ ]:
dtList = []

for i in range(len(power_df)):
    dtStr = power_df.iloc[i,0] + ' ' + str(power_df.iloc[i,1])+':00'
    dtList.append(dtStr)

power_df['일시'] = dtList
power_df

In [ ]:
for i, timeStr in enumerate(power_df['일시']):
    power_df.iloc[i,6] = (timeStr.replace('-','.'))

power_df

In [ ]:
delCol = ['거래일','거래시간','육지태양광 총발전량', '육지풍력 총발전량','제주풍력 총발전량']
power_df.drop(columns=delCol, inplace=True)
power_df['일시'] = pd.to_datetime(power_df['일시'])
power_df = power_df[power_df['일시']>'2020.03.1 8:00']
power_df.set_index(keys='일시', inplace=True, drop=True)
power_df

In [ ]:
saveFile='./태양광발전량.csv'
power_df.to_csv(saveFile, mode='w', encoding='utf-8-sig')

In [ ]:
first_df  = power_df[power_df.index<'2020-08-26 2:00:00'].shift(-1)
second_df = power_df[power_df.index>'2020-08-26 1:00:00']
power_df = pd.concat([first_df, second_df])
power_df

In [ ]:
saveFile='./solarGen1.csv'
power_df.to_csv(saveFile, mode='w', encoding='utf-8-sig')